# Chile Year Prediction, Yearly Features

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

c:\Users\marti\Desktop\data\hw_extra


In [3]:
def model_predict(data, regressor, len_pred, name_regressor=None, show=True):

    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data['Date'].dt.to_period('M').astype(str)

    indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
    features = data.columns.difference(["Date"]+ indices_of_interest)

    # Iterate over each index to create and evaluate models
    if show:
        fig, axs = plt.subplots(5, 1, figsize=(25,15))

    mae_errors = []
    mae_errors_training = []
    r2_scores = []
    r2_scores_training = []
    for i in range(len(indices_of_interest)):
        X = data[features]
        y = data[indices_of_interest[i]]
        dates = data["Date"]
        
        # Split into training and testing sets
        X_train, X_test, y_train, y_test = X[:-len_pred], X[-len_pred:], y[:-len_pred], y[-len_pred:]
        train_dates, test_dates = dates[:-len_pred], dates[-len_pred:]
        
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        y_pred_train = regressor.predict(X_train)
        
        # Evaluation
        mae_errors.append(mean_absolute_error(y_test, y_pred))
        mae_errors_training.append(mean_absolute_error(y_train, y_pred_train))
        r2_scores.append(r2_score(y_test, y_pred))
        r2_scores_training.append(r2_score(y_train, y_pred_train))
        
        if show:
            # Plot training values
            axs[i].plot(
                train_dates,
                y_train,
                label="Training",
                marker='o',
                color='green',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                train_dates,
                y_pred_train,
                label="Predicted Training",
                marker='x',
                color='red',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                test_dates,
                y_test,
                label="Test",
                marker='o',
                color='blue',
                linestyle='-',
                linewidth=1.5
            )
            axs[i].plot(
                test_dates,
                y_pred,
                label="Predicted Test",
                marker='x',
                color='red',
                linestyle='--',
                linewidth=1.5
            )
            axs[i].set_title(f"Prediction for {indices_of_interest[i]}")
            axs[i].legend()
    if show:
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig.suptitle(f"Model: {name_regressor}")
        plt.show()

    return [mae_errors, mae_errors_training, r2_scores, r2_scores_training]


In [4]:
data = pd.read_csv(f"data/features/chile/from1972/top10/HWs_chile_features_yearly.csv")

In [5]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
predictors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=10), RandomForestRegressor(random_state=42, n_estimators=50),  RandomForestRegressor(random_state=42, n_estimators=100), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1), XGBRegressor(random_state=42, n_estimators=50, learning_rate=0.1), XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_predictors = ["Linear", "RF10", "RF50", "RF100", "SVR-rbf", "XGB10",  "XGB50",  "XGB100", "GPR"]

In [6]:
stats = [model_predict(data, reg, 5, name_regressor=name, show=False) for reg,name in zip(predictors, name_predictors)]

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [7]:
indices_of_interest = ["Model", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_df = pd.DataFrame(columns=indices_of_interest)
for i in range(len(predictors)):
    results_df.loc[len(results_df.index)] = [f"{name_predictors[i]}"]+ stats[i][0]

results_df.style.set_caption("MAE Model Results in Prediction").format(precision=2)

,Model,HWN,HWF,HWD,HWM,HWA
0,Linear,3.17,12.64,2.28,1.10,3.20
1,RF10,2.07,8.89,1.37,0.92,2.53
2,RF50,2.03,8.40,1.29,0.79,2.60
3,RF100,2.01,8.18,1.25,0.78,2.59
4,SVR-rbf,2.22,8.53,1.46,0.78,2.24
5,XGB10,1.89,7.24,1.11,0.72,2.76
6,XGB50,1.89,7.06,1.33,0.79,3.27
7,XGB100,1.88,7.03,1.31,0.79,3.28
8,GPR,2.12,15.09,1.28,0.71,2.19


In [8]:
indices_of_interest = ["Model", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_df = pd.DataFrame(columns=indices_of_interest)
for i in range(len(predictors)):
    results_df.loc[len(results_df.index)] = [f"{name_predictors[i]}"]+ stats[i][1]

results_df.style.set_caption("MAE Model Results in Training").format(precision=2)

,Model,HWN,HWF,HWD,HWM,HWA
0,Linear,0.73,3.27,0.67,0.24,0.72
1,RF10,0.40,1.85,0.37,0.14,0.44
2,RF50,0.39,1.59,0.33,0.13,0.42
3,RF100,0.39,1.60,0.33,0.14,0.40
4,SVR-rbf,0.73,3.85,0.62,0.22,0.83
5,XGB10,0.54,2.24,0.47,0.18,0.56
6,XGB50,0.05,0.18,0.05,0.02,0.04
7,XGB100,0.00,0.01,0.00,0.00,0.00
8,GPR,0.94,7.84,0.86,0.34,1.16


In [9]:
indices_of_interest = ["Model", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_df = pd.DataFrame(columns=indices_of_interest)
for i in range(len(predictors)):
    results_df.loc[len(results_df.index)] = [f"{name_predictors[i]}"]+ stats[i][2]

results_df.style.set_caption("R^2 Model Results in Prediction").format(precision=2)

,Model,HWN,HWF,HWD,HWM,HWA
0,Linear,-11.23,-8.77,-8.55,-4.15,-4.62
1,RF10,-4.10,-3.28,-2.22,-1.86,-1.43
2,RF50,-3.54,-2.79,-1.68,-1.34,-1.62
3,RF100,-3.48,-2.61,-1.61,-1.25,-1.58
4,SVR-rbf,-4.14,-2.88,-2.56,-1.55,-1.07
5,XGB10,-2.79,-1.73,-1.27,-0.83,-1.79
6,XGB50,-2.94,-1.53,-1.98,-1.17,-2.74
7,XGB100,-2.86,-1.49,-1.94,-1.15,-2.76
8,GPR,-3.90,-9.25,-1.71,-1.07,-1.06


In [10]:
indices_of_interest = ["Model", "HWN", "HWF", "HWD", "HWM", "HWA"]
results_df = pd.DataFrame(columns=indices_of_interest)
for i in range(len(predictors)):
    results_df.loc[len(results_df.index)] = [f"{name_predictors[i]}"]+ stats[i][3]

results_df.style.set_caption("R^2 Model Results in Training").format(precision=2)

,Model,HWN,HWF,HWD,HWM,HWA
0,Linear,0.60,0.57,0.56,0.58,0.58
1,RF10,0.86,0.84,0.84,0.88,0.85
2,RF50,0.89,0.88,0.86,0.87,0.87
3,RF100,0.88,0.88,0.87,0.86,0.87
4,SVR-rbf,0.45,0.23,0.42,0.52,0.35
5,XGB10,0.76,0.77,0.74,0.77,0.76
6,XGB50,1.00,1.00,1.00,1.00,1.00
7,XGB100,1.00,1.00,1.00,1.00,1.00
8,GPR,0.22,-1.61,0.16,0.27,0.08
